In [23]:
import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.autograd import Variable

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from quantile_forest import RandomForestQuantileRegressor
from scipy.stats import iqr
from sklearn.linear_model import QuantileRegressor



import cvxpy as cp
import numpy as np
from numpy import linalg
import pandas as pd

from scipy.linalg import sqrtm


from matplotlib import pyplot as plt

import seaborn as sns
import matplotlib.patches as patches
import matplotlib.lines as lines

import ot

import warnings
random_seed = 42


if torch.cuda.is_available():
    print("GPU is available.")
    device = torch.device('cuda')
else:
    print("GPU is not available.")
    device = torch.device('cpu')

GPU is not available.


In [8]:
Sdata = pd.read_csv('ETTh1.csv', header = None)
SData = Sdata.to_numpy()
SData = SData[1:,1:]
SData = np.array(SData, dtype=float)
SData = SData[0:3000,:]

Tdata = pd.read_csv('ETTh2.csv', header = None)
TData = Tdata.to_numpy()
TData = TData[1:,1:]
TData = np.array(TData, dtype=float)
TData = TData[0:3000,:]

In [9]:
class NN1(nn.Module):
    def __init__(self, input_size, output_size):
        super(NN1, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)
        self.fc2 = nn.Linear(10, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# 2-layer NN
class NN2(nn.Module):
    def __init__(self, input_size, output_size):
        super(NN2, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 
    
    
    
class QuantileLoss(nn.Module):
    def __init__(self, quantile):
        super().__init__()
        self.quantile = quantile

    def forward(self, preds, target):
        assert not target.requires_grad
        assert preds.size(0) == target.size(0)
        residuals = target - preds
        return torch.max((self.quantile - 1) * residuals, self.quantile * residuals).mean()
        
        
def est_quantile(est_type,quantile,X_pre,Y_pre,X_opt,X_adj,X_t):
    """
        est_type: 
        "NN1": 1-layer NN;              "NN2": 2-layer NN; 
        "qrf": quantile regression forest;    "gb": gradient boosting
        
        quantile: the quantile we are estimating
        (X_pre,Y_pre): training data
        X_opt,X_adj,X_t: data used to predict
        output: quantile estimator Q and the prediction Q(X) 
    """
    
    num_var = X_pre.shape[1]
    
    if est_type == "NN1":
        torch.manual_seed(42)
        torch.cuda.manual_seed_all(42)
        
        # Convert numpy arrays to torch tensors
        X = torch.from_numpy(X_pre).float().to(device)
        Y = torch.from_numpy(Y_pre).float().to(device)
        
        model = NN1(input_size=num_var, output_size=1).to(device)
        learning_rate = 0.001
        
        # Set loss function and optimizer
        criterion = QuantileLoss(quantile).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        # Training loop
        for epoch in range(1000):
            optimizer.zero_grad()   # zero the gradient buffers
            output = model(X)
            loss = criterion(output, Y)
            loss.backward()
            optimizer.step()    # Does the update
            
        # Predict
        X_opt = torch.from_numpy(X_opt).float().to(device)
        X_adj = torch.from_numpy(X_adj).float().to(device)
        X_t = torch.from_numpy(X_t).float().to(device)

        Q_opt = model(X_opt)
        Q_opt = Q_opt.detach().cpu().numpy().reshape(-1,1)
        Q_adj = model(X_adj)
        Q_adj = Q_adj.detach().cpu().numpy().reshape(-1,1)
        Q_t = model(X_t)
        Q_t = Q_t.detach().cpu().numpy().reshape(-1,1)
        return model, Q_opt, Q_adj, Q_t
    
    if est_type == "NN2":
        torch.manual_seed(42)
        torch.cuda.manual_seed_all(42) 
        
        # Convert numpy arrays to torch tensors
        X = torch.from_numpy(X_pre).float().to(device)
        Y = torch.from_numpy(Y_pre).float().to(device)
        
        model = NN2(input_size=num_var, output_size=1).to(device)
        learning_rate = 0.001
        
        # Set loss function and optimizer
        criterion = QuantileLoss(quantile).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        # Training loop
        for epoch in range(1000):
            optimizer.zero_grad()   # zero the gradient buffers
            output = model(X)
            loss = criterion(output, Y)
            loss.backward()
            optimizer.step()    # Does the update
            
        # Predict
        X_opt = torch.from_numpy(X_opt).float().to(device)
        X_adj = torch.from_numpy(X_adj).float().to(device)
        X_t = torch.from_numpy(X_t).float().to(device)

        Q_opt = model(X_opt)
        Q_opt = Q_opt.detach().cpu().numpy().reshape(-1,1)
        Q_adj = model(X_adj)
        Q_adj = Q_adj.detach().cpu().numpy().reshape(-1,1)
        Q_t = model(X_t)
        Q_t = Q_t.detach().cpu().numpy().reshape(-1,1)
        return model, Q_opt, Q_adj, Q_t
    
    if est_type == "qrf":
        model = RandomForestQuantileRegressor(n_estimators = 500, random_state=random_seed)
        model.fit(X_pre, Y_pre)
        Q_opt = model.predict(X_opt,quantiles = [quantile]).reshape(-1,1)
        Q_adj = model.predict(X_adj,quantiles = [quantile]).reshape(-1,1)
        Q_t = model.predict(X_t,quantiles = [quantile]).reshape(-1,1)
        return model, Q_opt, Q_adj, Q_t
    
    
    if est_type == "gb":
        model = GradientBoostingRegressor(n_estimators=300,random_state=random_seed,loss = "quantile", alpha = quantile)
        model.fit(X_pre, Y_pre)
        Q_opt = model.predict(X_opt).reshape(-1,1)
        Q_adj = model.predict(X_adj).reshape(-1,1)
        Q_t = model.predict(X_t).reshape(-1,1)
        return model, Q_opt, Q_adj, Q_t


In [10]:
def run_utopia(mc_iter):
    Dtrain = SData
    Dshift = TData
    Y_train = Dtrain[:,-1]
    X_train = Dtrain[:,:-1]

    X_test = Dshift[:,:-1]
    Y_test = Dshift[:,-1]


    pre_idx = int(Dtrain.shape[0] * 0.5)
    Dtrain_pre, Dtrain_opt = np.split(Dtrain, [pre_idx])
    adj_idx = int(Dtrain_opt.shape[0] * 0.5)
    Dtrain_opt, Dtrain_adj = np.split(Dtrain_opt, [adj_idx])

    X_pre = Dtrain_pre[:,:-1]
    Y_pre = Dtrain_pre[:,-1]

    X_opt = Dtrain_opt[:,:-1]
    Y_opt = Dtrain_opt[:,-1]

    X_adj = Dtrain_adj[:,:-1]
    Y_adj = Dtrain_adj[:,-1]
    
    #mean_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    mean_model = LinearRegression()
    mean_model.fit(X_pre, Y_pre)

    mean_est_pre = mean_model.predict(X_pre)
    mean_est_opt = mean_model.predict(X_opt)
    mean_est_adj = mean_model.predict(X_adj)

    Y_centered_squared_pre = (Y_pre - mean_est_pre)**2 
    Y_centered_squared_opt = (Y_opt - mean_est_opt)**2 
    Y_centered_squared_adj = (Y_adj - mean_est_adj)**2 
    
    
    ######## Estimation of the shift ###########
#     ot_sinkhorn = ot.da.SinkhornTransport(reg_e=1e-1, max_iter=1000)
#     ot_sinkhorn.fit(Xs=X_test, Xt=X_pre)
#     X_test = ot_sinkhorn.transform(Xs=X_test)
    
    ot_emd = ot.da.EMDTransport()
    ot_emd.fit(Xs=X_test, Xt=X_pre)
    X_test = ot_emd.transform(Xs=X_test)
    
#     ot_emd = ot.da.LinearTransport()
#     ot_emd.fit(Xs=X_test, Xt=X_pre)
#     X_test = ot_emd.transform(Xs=X_test)
    
    
    quantile = [0.85,0.95,0.9,0.9]


    m1,Q1_opt,Q1_adj,Q1_t = est_quantile("NN1",quantile[0],X_pre,Y_centered_squared_pre,X_opt,X_adj,X_test) # quantile def
    m2,Q2_opt,Q2_adj,Q2_t = est_quantile("NN2",quantile[1],X_pre,Y_centered_squared_pre,X_opt,X_adj,X_test)
    m3,Q3_opt,Q3_adj,Q3_t = est_quantile("qrf",quantile[2],X_pre,Y_centered_squared_pre,X_opt,X_adj,X_test)
    m4,Q4_opt,Q4_adj,Q4_t = est_quantile("gb",quantile[3],X_pre,Y_centered_squared_pre,X_opt,X_adj,X_test)
    

    ######## Variance estimator ###########
    
    rf_var_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    rf_var_model.fit(X_pre, Y_centered_squared_pre)
    var_hat_pre = rf_var_model.predict(X_pre)
    var_hat_adj = rf_var_model.predict(X_adj)
    var_hat_opt = rf_var_model.predict(X_opt)
    var_hat_test = rf_var_model.predict(X_test)
    
    E_opt = np.vstack((np.matrix.flatten(Q1_opt), np.matrix.flatten(Q2_opt), np.matrix.flatten(Q3_opt), np.matrix.flatten(Q4_opt), var_hat_opt))
    E_adj = np.vstack((np.matrix.flatten(Q1_adj), np.matrix.flatten(Q2_adj), np.matrix.flatten(Q3_adj), np.matrix.flatten(Q4_adj), var_hat_adj))
    E_test = np.vstack((np.matrix.flatten(Q1_t), np.matrix.flatten(Q2_t), np.matrix.flatten(Q3_t), np.matrix.flatten(Q4_t), var_hat_test))
    
    n_opt = X_opt.shape[0]
    n_adj = X_adj.shape[0]
    n_test = X_test.shape[0]

    cons_opt = np.ones(n_opt).reshape(1,-1)
    cons_adj = np.ones(n_adj).reshape(1,-1)
    cons_test = np.ones(n_test).reshape(1,-1)

    A_opt = np.vstack((E_opt,cons_opt))
    A_adj = np.vstack((E_adj,cons_adj))
    A_test = np.vstack((E_test,cons_test))

    weight = cp.Variable(A_opt.shape[0])


    constraints = [weight>=0]+[weight @ A_opt >= Y_centered_squared_opt]  ### Change the objective 
    prob = cp.Problem(cp.Minimize(cp.sum(weight @ A_opt)), constraints)
    prob.solve()
    optimal_weight = weight.value

    f_hat_init_opt = optimal_weight @  A_opt
    f_hat_init_adj = optimal_weight @  A_adj
    f_hat_init_test = optimal_weight @ A_test

        
    
    alpha = 0.05
    scores_temp = Y_centered_squared_adj/f_hat_init_adj
    level = (1 - alpha) * 100
    level = int(level)
    delta = np.percentile(scores_temp, level)
    
    
    mean_est_test = mean_model.predict(X_test)
    Y_centered_squared_test = (Y_test - mean_est_test)**2 
    cov = np.mean(Y_centered_squared_test < (delta * f_hat_init_test))
    bw = 2 * np.mean((delta * f_hat_init_test) ** 0.5)
    return cov, bw, delta

In [11]:
cov,bw,delta = run_utopia(1)
print(cov)
print(bw)

/Users/mdeb/anaconda3/lib/python3.10/site-packages/ot/lp/__init__.py:361: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
/Users/mdeb/anaconda3/lib/python3.10/site-packages/ot/da.py:594: RuntimeWarning: invalid value encountered in divide
  transp = self.coupling_ / nx.sum(self.coupling_, axis=1)[:, None]


0.976
41.524866061296855


In [12]:
############################# Weighted variance adjusted split conformal #######################################

def prob_est(Dtrain, Dshift, est_type = 'lr'):
    X_S = Dtrain[:,:-1]
    X_T = Dshift[:,:-1]
    n_S = X_S.shape[0]
    n_T = X_T.shape[0]
    Y_S = np.zeros(n_S).reshape(-1,1)
    Y_T = np.ones(n_T).reshape(-1,1)
    X_train = np.vstack((X_S, X_T))
    Y_train = np.vstack((Y_S, Y_T))
  
    if est_type == 'lr':
        # fit logistic regression
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            logistic_model = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000,random_state=0) 
            logistic_model.fit(X_train, Y_train[:,0])

        return logistic_model
    
    if est_type == 'rf':
        # fit random forest
        rf_model = RandomForestClassifier(n_estimators = 100, random_state=random_seed)
        rf_model.fit(X_train, Y_train[:,0])

        return rf_model

    
def ratio_est(model, X):
    
        prob_1 = model.predict_proba(X)[:,1]
        prob_1 = np.clip(prob_1, 0.01, 0.99)
        est_ratio = prob_1/(1-prob_1)
        
        return est_ratio
    
def weighting_function(model, X_calib, x_test, x):
    # Define weighting function here
    # all the data should be array
    ratio_calib = ratio_est(model, X_calib).sum()
    ratio_test = ratio_est(model, x_test)
    ratio_sum = ratio_calib + ratio_test
    ratio_x = ratio_est(model, x)
    return ratio_x/ratio_sum


def weighted_quantile(values, weights, quantile):
    
    """ Compute the weighted quantile of a 1D numpy array.
    """
    values = np.array(values)
    weights = np.array(weights)
    
    sorter = np.argsort(values)
    values = values[sorter]
    weights = weights[sorter]
    cumulative_weights = np.cumsum(weights)
    if cumulative_weights[-1]>= quantile:
        idx = np.argmax(cumulative_weights >= quantile)
        return values[idx]
    else:
        return np.max(values)


In [13]:
def weighted_conformal_cov_width(Dtrain, Dshift, alpha=0.05):
    
    n = Dshift.shape[0]
    cov = []
    width = []
    
    
    prob_est_model = prob_est(Dtrain, Dshift, est_type = 'lr')
    
    # split weighted conformal
    D = Dtrain
    np.random.shuffle(D)
    split_idx = int(D.shape[0] * 0.5)
    D1, D2 = np.split(D, [split_idx])
    X1 = D1[:,:-1]
    y1 = D1[:,-1]
    X2 = D2[:,:-1]
    y2 = D2[:,-1]
    
    # Train the model on the first part of the data
    model = LinearRegression()
    model.fit(X1, y1)
    
    # Compute the model's predictions for the calibration set
    y1_hat = model.predict(X1)
    y_calib_pred = model.predict(X2)
            
    # Calculate the absolute errors on the calibration set
    errors_calib = np.abs(y_calib_pred - y2)
    
    est_error_1 = (y1_hat - y1) ** 2
    var_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    var_model.fit(X1, est_error_1)
    var_hat = var_model.predict(X2)
    sd_calib_pred = var_hat ** 0.5
    errors_calib = np.abs(y_calib_pred - y2)/np.clip(sd_calib_pred, 0.001, 1e10)
    
    for i in range(n):
        x_test = Dshift[i,:-1]
        x_test = np.array(x_test).reshape(1,-1)
        y_test = Dshift[i,-1]
    
    
        # Calculate the weights for the calibration set
        weights_calib = weighting_function(prob_est_model, X2, x_test, X2)


        # Calculate the weighted quantile of the errors
        quantile = weighted_quantile(errors_calib, weights_calib, 1 - alpha)

        # Now for the test
        y_test_pred = model.predict(x_test)
        sd_test_pred = var_model.predict(x_test) ** 0.5
        
        width.append(2*quantile*sd_test_pred)
        cov.append((np.abs(y_test - y_test_pred)/sd_test_pred) <= quantile)

    
    return np.array(cov).mean(), np.array(width).mean()


In [14]:
alpha = 0.05
def run_weighted_conformal(n):
    Dtrain = SData
    Dshift = TData
    cov, bw = weighted_conformal_cov_width(Dtrain, Dshift, alpha)
    return cov, bw

In [15]:
cov, bw = run_weighted_conformal(1)
print(cov)
print(bw)

0.9816666666666667
57.90698316455262


In [28]:
def weighted_conformal_quantile(Dtrain, Dshift, alpha=0.05):
    
    n = Dshift.shape[0]
    cov = []
    width = []
    
    
    prob_est_model = prob_est(Dtrain, Dshift, est_type = 'lr')
    
    # split weighted conformal
    D = Dtrain
    np.random.shuffle(D)
    split_idx = int(D.shape[0] * 0.5)
    D1, D2 = np.split(D, [split_idx])
    X1 = D1[:,:-1]
    y1 = D1[:,-1]
    X2 = D2[:,:-1]
    y2 = D2[:,-1]
    
    # Train the model on the first part of the data
#     model_quantile = RandomForestQuantileRegressor(n_estimators = 500, random_state=random_seed)
#     model_quantile.fit(X1, y1)
    
    qr_lb = QuantileRegressor(quantile=alpha/2, solver = 'highs')
    qr_ub = QuantileRegressor(quantile = (1-alpha)/2, solver = 'highs')
    
    qr_lb.fit(X1, y1)
    qr_ub.fit(X1, y1)
    
#     y2_lb_hat = model_quantile.predict(X2, quantiles = alpha/2)
#     y2_ub_hat = model_quantile.predict(X2, quantiles = (1-alpha)/2)
    
    y2_lb_hat = qr_lb.predict(X2)
    y2_ub_hat = qr_ub.predict(X2)
        
#     model_ub = QuantileRegressor(quantile = (1-alpha)/2, alpha = 1, fit_intercept=True, solver='highs', solver_options=None)
#     model_ub.fit(X1, y1)
    
#     model_lb = QuantileRegressor(quantile = alpha/2, alpha = 1, fit_intercept=True, solver='highs', solver_options=None)
#     model_lb.fit(X1, y1)
    
#     y2_lb_hat = model_lb.predict(X2)
#     y2_ub_hat = model_ub.predict(X2)
    
    errors_calib = np.maximum(y2_lb_hat - y2, y2 - y2_ub_hat)
    
    for i in range(n):
        x_test = Dshift[i,:-1]
        x_test = np.array(x_test).reshape(1,-1)
        y_test = Dshift[i,-1]
    
    
        # Calculate the weights for the calibration set
        weights_calib = weighting_function(prob_est_model, X2, x_test, X2)


        # Calculate the weighted quantile of the errors
        quantile = weighted_quantile(errors_calib, weights_calib, 1 - alpha)
        
#         y_test_lb = model_quantile.predict(x_test, quantiles = alpha/2)
#         y_test_ub = model_quantile.predict(x_test, quantiles = (1-alpha)/2)
        
        y_test_lb = qr_lb.predict(x_test)
        y_test_ub = qr_ub.predict(x_test)
        
        width.append(2*quantile + y_test_ub - y_test_lb)
        cov.append(y_test >= y_test_lb - quantile and y_test <= y_test_ub + quantile)

    
    return np.array(cov).mean(), np.array(width).mean()


In [29]:
alpha = 0.05
def run_weighted_conformal_quantile(n):
    Dtrain = SData
    Dshift = TData
    cov, bw = weighted_conformal_quantile(Dtrain, Dshift, alpha)
    return cov, bw


In [30]:
cov, bw = run_weighted_conformal_quantile(1)
print(cov)
print(bw)

0.8416666666666667
54.98122101338704
